In [1]:
import exchange_calendars as xcals
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb
from sqlalchemy import create_engine, text
from sqlalchemy.exc import ProgrammingError

In [2]:
obb.user.preferences.output_type = "dataframe"

In [3]:
username = "myuser"
password = "mypassword"
host = "127.0.0.1"
port = "5432"
database = "mydb"

In [4]:
DATABASE_URL = f"postgresql://{username}:{password}@{host}:{port}/postgres"
base_engine = create_engine(DATABASE_URL)

In [5]:
def create_database_and_get_engine(db_name, base_engine):
    conn = base_engine.connect()
    conn = conn.execution_options(isolation_level="AUTOCOMMIT")

    try:
        conn.execute(text(f"CREATE DATABASE {db_name};"))
    except ProgrammingError:
        pass
    finally:
        conn.close()

    conn_str = base_engine.url.set(database=db_name)

    return create_engine(conn_str)

In [10]:
engine = create_database_and_get_engine("stock_data", base_engine)


In [11]:
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol=symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

In [12]:
def save_data_range(symbol, engine, start_date=None, end_date=None):
    data = get_stock_data(symbol, start_date, end_date)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

In [14]:

def save_last_trading_session(symbol, engine):
    today = pd.Timestamp.today()
    data = get_stock_data(symbol, today, today)
    data.to_sql("stock_data", engine, if_exists="append", index=False)

In [15]:
for symbol in ["SPY", "QQQ", "DIA"]:
    save_data_range(
        symbol, engine=engine, start_date="2020-06-01", end_date="2023-01-01"
    )

In [18]:
df_1 = pd.read_sql_query("SELECT * from stock_data where symbol='SPY'", engine)
display(df_1)

,date,open,high,low,close,volume,split_ratio,dividend,capital_gains,symbol
0,2020-06-01,303.619995,306.209991,303.059998,305.549988,55758300,0.0,0.0,0.0,SPY
1,2020-06-02,306.549988,308.130005,305.100006,308.079987,74267200,0.0,0.0,0.0,SPY
2,2020-06-03,310.239990,313.220001,309.940002,312.179993,92567600,0.0,0.0,0.0,SPY
3,2020-06-04,311.109985,313.000000,309.079987,311.359985,75794400,0.0,0.0,0.0,SPY
4,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.0,0.0,0.0,SPY
...,...,...,...,...,...,...,...,...,...,...
648,2022-12-23,379.649994,383.059998,378.029999,382.910004,59857300,0.0,0.0,0.0,SPY
649,2022-12-27,382.790009,383.149994,379.649994,381.399994,51638200,0.0,0.0,0.0,SPY
650,2022-12-28,381.329987,383.390015,376.420013,376.660004,70911500,0.0,0.0,0.0,SPY
651,2022-12-29,379.630005,384.350006,379.079987,383.440002,66970900,0.0,0.0,0.0,SPY


In [19]:
df_2 = pd.read_sql_query(
    "SELECT * from stock_data where symbol='SPY' and volume > 100000000", engine
)
display(df_2)

,date,open,high,low,close,volume,split_ratio,dividend,capital_gains,symbol
0,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.0,0.000,0.0,SPY
1,2020-06-11,311.459991,312.149994,300.010010,300.609985,209243600,0.0,0.000,0.0,SPY
2,2020-06-12,308.239990,309.079987,298.600006,304.209991,194678900,0.0,0.000,0.0,SPY
3,2020-06-15,298.019989,308.279999,296.739990,307.049988,135782700,0.0,0.000,0.0,SPY
4,2020-06-16,315.480011,315.640015,307.670013,312.959991,137627500,0.0,0.000,0.0,SPY
...,...,...,...,...,...,...,...,...,...,...
149,2022-12-13,410.220001,410.489990,399.070007,401.970001,123782500,0.0,0.000,0.0,SPY
150,2022-12-14,401.609985,405.500000,396.309998,399.399994,108111300,0.0,0.000,0.0,SPY
151,2022-12-15,394.299988,395.250000,387.890015,389.630005,117705900,0.0,0.000,0.0,SPY
152,2022-12-16,385.179993,386.579987,381.040009,383.269989,119858000,0.0,1.781,0.0,SPY
